In [1]:
import numpy as np
import cv2
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import salientregions as sr

In [2]:
#Load the image
path_to_image = '/home/dafne/SalientRegions/LargeScaleImaging/Data/OxFrei/01_graffiti/JPG/_original.jpg'
#path_to_image = '../../../Data/OxFrei/09_small_palace/_original.jpg'
original = cv2.imread(path_to_image)
sr.show_image(original)
path_to_image = '/home/dafne/SalientRegions/LargeScaleImaging/Data/OxFrei/01_graffiti/JPG/viewpoint1.jpg'
viewpoint = cv2.imread(path_to_image)
sr.show_image(viewpoint)

In [3]:
#Convert to grey scale
grayscale = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
grayscale1 = cv2.cvtColor(viewpoint, cv2.COLOR_BGR2GRAY)
sr.show_image(grayscale)
sr.show_image(grayscale1)

In [4]:
#Binarize according to threshold
ret, binarized = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
ret, binarized1 = cv2.threshold(grayscale1, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
sr.show_image(binarized)
sr.show_image(binarized1)

In [7]:
lam_large = 0.4
lam_verylarge = 0.7
for t in xrange(256) :
    #bint = sr.binarize(grayscale, threshold=t)
    

IndentationError: expected an indented block (<ipython-input-7-d412ccfa5c5a>, line 5)

In [40]:
holes, islands, indents, prots = sr.get_salient_regions(binarized, visualize=False)
holes1, islands1, indents1, prots1 = sr.get_salient_regions(binarized1, visualize=False)

In [6]:
contour_image = holes.copy()
sr.show_image(contour_image)

In [7]:
#img_contour, contours, hierarchy 
contour_image = holes.copy()
img, contours, hierarchy = cv2.findContours(contour_image,cv2.RETR_LIST ,cv2.CHAIN_APPROX_SIMPLE)
contour_image =  np.zeros(contour_image.shape, dtype='uint8')
contour_image = cv2.drawContours(contour_image, [contours[2]], -1, (128, 255, 0), 1)
sr.show_image(contour_image)

In [ ]:
#Find the connected components (similar to bwlabel)
retval, labels = cv2.connectedComponents(binarized[:,:,0]) 
#An ugly transformation to see the CC's in different colors
dim = np.ceil(np.power(retval, 1.0/3))
step = int(np.floor(255/dim))
print dim, step
rvalues = np.arange(int(np.floor(step/2)),256, step)
colormapping = dict([(i,[rvalues[np.mod(i, dim)], 
                         rvalues[np.floor(np.mod(i, dim*dim)/dim)], 
                         rvalues[np.floor(i/dim)]]) for i in xrange(1, retval)])
colormapping[0] = [0,0,0]
colored_labels = np.array([[colormapping[i] for i in x]  for x in labels], dtype='uint8')
show_image(colored_labels)

In [7]:
sr.visualize_elements(binarized, holes, islands, indents, prots)
print ''

In [46]:
#Get keypoints for the salient regions, from cc
def get_keypoints(mask):
    nccs, labels, stats, centroids = cv2.connectedComponentsWithStats(mask)
    kps = []
    for i in xrange(1, nccs):
        x,y = centroids[i]
        area = stats[i][cv2.CC_STAT_AREA]
        width = stats[i][cv2.CC_STAT_WIDTH]
        height = stats[i][cv2.CC_STAT_HEIGHT]
        diam = max(width, height) #2 * np.sqrt(area/(np.pi))
        kps.append(cv2.KeyPoint(x, y, _size=diam))
    return kps

In [47]:
#Get Keypoints for our salient regions
sr_kps = get_keypoints(holes) + get_keypoints(islands) + get_keypoints(indents) + get_keypoints(prots)
sr_kps1 = get_keypoints(holes1) + get_keypoints(islands1) + get_keypoints(indents1) + get_keypoints(prots1)

In [17]:
#Try out MSR
vis =  cv2.cvtColor(grayscale.copy(), cv2.COLOR_GRAY2BGR)
mser = cv2.MSER_create()
regions = mser.detectRegions(grayscale, None)
region_kps = mser.detect(grayscale)
region_kps1 = mser.detect(grayscale1)
hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
cv2.polylines(vis, hulls, 1, (0, 255, 0), 1)
sr.show_image(vis, 'MSER regions')

In [79]:
sr.show_image(vis, 'MSER regions')
len(region_kps1)

2099

In [49]:
orb = cv2.ORB_create()
orb_kps, orb_des = orb.compute(grayscale, sr_kps)
orb_kps1, orb_des1 = orb.compute(grayscale1, sr_kps1)

In [50]:
#Matching!
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(orb_des, orb_des1)
matches = sorted(matches, key = lambda x:x.distance)
img3 = cv2.drawMatches(grayscale,orb_kps,grayscale1,orb_kps1,matches[:10], None, flags=2, )

In [51]:
sr.show_image(img3, 'matches', 100000)

In [48]:
im_with_keypoints = cv2.drawKeypoints(grayscale, sr_kps, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sr.show_image(im_with_keypoints, 'keypoints', 100000)

In [45]:
?cv2.drawMatches